In [1]:
from xgboost import XGBClassifier
import pprint
import operator
import collections
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, accuracy_score

Using TensorFlow backend.


In [242]:
file = '/Users/arpitjain/Desktop/changeClassifierResult/ASSET_change_CC_FormalEMP&Lit_XGB_WO_2001.pkl'
tss = pickle.load(open(file,'rb'))
print('val_f1_score ',tss['xgBoost']['val_scores'])
print('train_f1_score ',tss['xgBoost']['train_scores'])

val_f1_score  [0.7481875244188912, 0.7428176612907225, 0.7311546249964802, 0.7143782299395331, 0.7049173928782612]
train_f1_score  [0.7937083809252552, 0.7885771678454196, 0.7984200753961589, 0.788575695719303, 0.7903365189203497]


In [243]:
n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
max_depth = tss['xgBoost']['specs'][0]['max_depth']
learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
objective = tss['xgBoost']['specs'][0]['objective']
booster = tss['xgBoost']['specs'][0]['booster']
gamma = tss['xgBoost']['specs'][0]['gamma']
min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
subsample = tss['xgBoost']['specs'][0]['subsample']
colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = tss['xgBoost']['specs'][0]['base_score']

n_splits = tss['xgBoost']['specs'][0]['kFold_splits']

print('                 ')
print('Re-Training again for calculating f1 and train scores')

xgbc = XGBClassifier(n_estimators=n_estimators, 
         max_depth=max_depth, learning_rate=learning_rate, 
         objective=objective, booster=booster,n_jobs=-1, 
         gamma=gamma, min_child_weight=min_child_weight, 
         max_delta_step=max_delta_step, subsample=subsample, 
         colsample_bytree=colsample_bytree, 
         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

cv = KFold(n_splits=n_splits, shuffle=True)

                 
Re-Training again for calculating f1 and train scores


## Data 

In [244]:
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/GroundTruth_ChangeClassifier.csv")
df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/ASSET_CC.csv")
data = ground_truth.merge(df, on='census_code', how='left')
data.dropna(inplace=True)
target = 'ASSET_change'
feature_cols = ['Formal_EMP_2001','ASSET_2001','FC_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009','predictions_2011']

In [245]:
ground_truth.head(2)

,census_code,MSW_change,MSW_2001,BF_change,BF_2001,CHH_change,CHH_2001,EMP_change,EMP_2001,MSL_change,MSL_2001,FC_change,FC_2001,ASSET_change,ASSET_2001,Formal_EMP_2001,LIT_2001
0,1,0,1,0,2,0,1,0,1,1,2,0,1,0,1,2,1
1,2,0,3,0,2,1,1,0,1,0,3,0,2,1,1,2,1


In [246]:
X = data[feature_cols].values
y = data[target].values

In [247]:
X.shape

(580, 8)

In [248]:
y.shape

(580,)

In [249]:
val_f1score = []
val_accscore = []
train_f1score = []
train_accscore = []

test_prediction_list = []
index_list = []
# f1score = []
# accscore = []
counter=0
for train_index, test_index in cv.split(X):
    print(counter)
    counter=counter+1
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    best_xgb = xgbc.fit(X_train, y_train)
    
#     prediction_2011 = best_xgb.predict(X_2011)
#     f1_weight = f1_score(y_2011, prediction_2011, average='weighted')
#     acc = accuracy_score(y_2011, prediction_2011)

#     f1score.append(f1_weight)
#     accscore.append(acc)

    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)
    
    test_prediction_list.append(predictions)
    index_list.append(test_index)

    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    acc = accuracy_score(y_test, predictions)
    acc_train = accuracy_score(y_train, predictions_train)

    val_f1score.append(f1_weight)
    train_f1score.append(f1_weight_train)
    val_accscore.append(acc)
    train_accscore.append(acc_train)

val_f1score = np.array(val_f1score).mean()
train_f1score = np.array(train_f1score).mean()
val_accscore = np.array(val_accscore).mean()
train_accscore = np.array(train_accscore).mean()
# f1score = np.array(f1score).mean()
# accscore = np.array(accscore).mean()
# print(f1score, accscore)
print(val_f1score, train_f1score, val_accscore, train_accscore)

0
1
2
3
4
0.7568516733285029 0.789690577151252 0.7603448275862069 0.7916026322844253


In [250]:
census_codes = data['census_code'].values

In [251]:
df_list = []
for i in range(len(index_list)):
    a = census_codes[index_list[i]]
    b = test_prediction_list[i]
    c = np.vstack((a,b))
    df = pd.DataFrame(c).transpose()
    df_list.append(df)

In [252]:
final_df = pd.concat(df_list, axis=0)

In [253]:
final_df.columns = ['census_code', 'change']

In [254]:
final_df.shape

(580, 2)

In [255]:
final_df.head()

,census_code,change
0,4,1
1,10,1
2,14,1
3,24,1
4,27,1


In [256]:
final_df.drop_duplicates(subset='census_code',inplace=True)

In [257]:
final_df.shape

(576, 2)

In [258]:
final_df.to_csv('ASSET_Change.csv')